In [1]:
import csv
from datetime import datetime

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import pandas as pd
import re

import statsmodels.api as sm
import statsmodels.formula.api as smf
import patsy

from IPython.display import Image
import seaborn as sns

C:\Users\Arina\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
# pandas display settings

pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 50)
pd.set_option('display.precision', 3)

### 1. Read in the data

In [3]:
ls ..\data_local

 Volume in drive C is Windows
 Volume Serial Number is 0AC5-9A85

 Directory of C:\Users\Arina\data_science\metis\project3\data_local

02/08/2018  08:00 PM    <DIR>          .
02/08/2018  08:00 PM    <DIR>          ..
02/08/2018  03:58 PM        26,409,403 whales_encounters.json
02/07/2018  02:21 PM           165,343 whales_encounters_subset1.json
02/07/2018  05:44 PM         4,066,830 whales_encounters_subset2.json
02/08/2018  03:32 PM        25,810,202 whales_encounters_subset3.json
               4 File(s)     56,451,778 bytes
               2 Dir(s)  586,978,291,712 bytes free


In [4]:
filepath = '..\data_local\whales_encounters.json'
df = pd.read_json(filepath, dtype = False)
df.head()

,additional_comments,beh_role,behavior,encounter_date,encounter_id,encounter_location_info,encounter_sex,est_size_best,group_behavior,group_type,individual_id,life_stage,locality,number_calves,occurrence_id,photographer,pos_type,region_name,scarring,sighting,sighting_comments,status,submitter,url,vessel
0,"[ \r\n\r\n, \r\n \r\n\r\n]",[],"[ , \r\n \r\n Solo\r\n \r\n]",\r\n 2004-8-16 9:17\r\n,[: 10059\r\n ],"[ British Columbia\r\n \r\n , \r\n\r\n , : ...",male \r\n\r\n,"[ , 1\r\n \r\n]","[ , Fluke Slap\r\n \r\n]","[ , Single\r\n \r\n]",460044\r\n,"[ , \r\n \r\n]","[ , Ikeda Inlet to Juan Perez Sound\r\n \r\n]","[ , 0\r\n \r\n]",DFOC:Mon Aug 16 00:00:00 EDT 2004:N2:2,"[ \r\n \t, John Calambokidis \r\n , \r\n\r\n...","[ , Precise\r\n \r\n]","[ , Northern British Columbia\r\n \r\n]",\r\n\r\nNone\r\n \r\n\r\n,2\r\n \r\n,"[ , TAIL LOBBING\r\n \r\n]",\r\n Status: alive \r\n,"[ \r\n \r\n , Department of Fisheries and Oce...",http://www.splashcatalog.org:80/encounters/enc...,"[ , N2\r\n \r\n]"
1,"[ \r\n\r\n, \r\n \r\n\r\n]",[],"[ , \r\n \r\n Unspecified competitive group ...",\r\n 2004-3-29 12:53\r\n,[: 1006\r\n ],"[ Baja California\r\n \r\n , \r\n\r\n , : M...",male \r\n\r\n,"[ , 6\r\n \r\n]","[ , Competitive\r\n \r\n]","[ , Competitive Group\r\n \r\n]",420485\r\n,"[ , \r\n \r\n]","[ , Los Frailes\r\n \r\n]","[ , 0\r\n \r\n]",UABCS:Mon Mar 29 00:00:00 EST 2004:RA:6,"[ \r\n \t, \r\n , \r\n\r\n\r\n]","[ , Precise\r\n \r\n]","[ , Mexico- Baja Calfiornia\r\n \r\n]",\r\n\r\nNone\r\n \r\n\r\n,6\r\n \r\n,[],\r\n Status: alive \r\n,"[ \r\n \r\n , Universidad Autonoma de Baja Ca...",http://www.splashcatalog.org:80/encounters/enc...,"[ , RA\r\n \r\n]"
2,"[ \r\n\r\n, \r\n \r\n\r\n]","[ , AD\r\n \r\n]","[ , \r\n \r\n Unknown\r\n \r\n]",\r\n 2004-8-26 9:00\r\n,[: 10056\r\n ],"[ Kodiak Island\r\n \r\n , \r\n\r\n , : NGO...",male \r\n\r\n,"[ , 40\r\n \r\n]","[ , Feeding\r\n \r\n]","[ , UNKN\r\n \r\n]",472177\r\n,"[ , \r\n \r\n]","[ , KODIAK\r\n \r\n]","[ , 1\r\n \r\n]",UAFK:Thu Aug 26 00:00:00 EDT 2004:DU:1,"[ \r\n \t, \r\n , \r\n\r\n\r\n]","[ , Precise\r\n \r\n]","[ , Northern Gulf of Alaska\r\n \r\n]",\r\n\r\nNone\r\n \r\n\r\n,1\r\n \r\n,"[ , large feeding aggregation_ loose affilia...",\r\n Status: alive \r\n,"[ \r\n \r\n , University of Alaska, Fairbanks...",http://www.splashcatalog.org:80/encounters/enc...,"[ , DU\r\n \r\n]"
3,"[ \r\n\r\n, \r\n \r\n\r\n]","[ , UN\r\n \r\n]","[ , \r\n \r\n Unknown\r\n \r\n]",\r\n 2004-8-26 9:00\r\n,[: 10055\r\n ],"[ Kodiak Island\r\n \r\n , \r\n\r\n , : NGO...",unknown \r\n\r\n,"[ , 40\r\n \r\n]","[ , Feeding\r\n \r\n]","[ , UNKN\r\n \r\n]",472380\r\n,"[ , \r\n \r\n]","[ , KODIAK\r\n \r\n]","[ , 1\r\n \r\n]",UAFK:Thu Aug 26 00:00:00 EDT 2004:DU:1,"[ \r\n \t, \r\n , \r\n\r\n\r\n]","[ , Precise\r\n \r\n]","[ , Northern Gulf of Alaska\r\n \r\n]",\r\n\r\nNone\r\n \r\n\r\n,1\r\n \r\n,"[ , large feeding aggregation_ loose affilia...",\r\n Status: alive \r\n,"[ \r\n \r\n , University of Alaska, Fairbanks...",http://www.splashcatalog.org:80/encounters/enc...,"[ , DU\r\n \r\n]"
4,"[ \r\n\r\nMom?, \r\n \r\n\r\n]","[ , G3\r\n \r\n]","[ , \r\n \r\n Trio, non-competitive\r\n \r\n]",\r\n 2004-9-25 16:20\r\n,[: 10044\r\n ],"[ California\r\n \r\n , \r\n\r\n , : CA-OR\...",unknown \r\n\r\n,"[ , 3\r\n \r\n]",[],"[ , Mom calf Escort\r\n \r\n]",450262\r\n,"[ , \r\n \r\n]","[ , N. Cordell Bank\r\n \r\n]","[ , 0?\r\n \r\n]",CRC:Sat Sep 25 00:00:00 EDT 2004:N2:6,"[ \r\n \t, John Calambokidis \r\n , \r\n\r\n...","[ , Precise\r\n \r\n]","[ , California-Oregon\r\n \r\n]",\r\n\r\nNone\r\n \r\n\r\n,6\r\n \r\n,[],\r\n Status: alive \r\n,"[ \r\n \r\n , Cascadia Research Collective\r\...",http://www.splashcatalog.org:80/encounters/enc...,"[ , N2\r\n \r\n]"


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19185 entries, 0 to 19184
Data columns (total 25 columns):
additional_comments        19185 non-null object
beh_role                   19185 non-null object
behavior                   19185 non-null object
encounter_date             19185 non-null object
encounter_id               19185 non-null object
encounter_location_info    19185 non-null object
encounter_sex              19185 non-null object
est_size_best              19185 non-null object
group_behavior             19185 non-null object
group_type                 19185 non-null object
individual_id              19185 non-null object
life_stage                 19185 non-null object
locality                   19185 non-null object
number_calves              19185 non-null object
occurrence_id              19185 non-null object
photographer               19185 non-null object
pos_type                   19185 non-null object
region_name                19185 non-null object
scarring 

In [6]:
df.shape

(19185, 25)

In [7]:
# make a copy of the df, because reading in json takes a while
df2 = df.copy()

In [8]:
df2.columns

Index(['additional_comments', 'beh_role', 'behavior', 'encounter_date',
       'encounter_id', 'encounter_location_info', 'encounter_sex',
       'est_size_best', 'group_behavior', 'group_type', 'individual_id',
       'life_stage', 'locality', 'number_calves', 'occurrence_id',
       'photographer', 'pos_type', 'region_name', 'scarring', 'sighting',
       'sighting_comments', 'status', 'submitter', 'url', 'vessel'],
      dtype='object')

### 2. Clean raw data fields

In [9]:
# list-to-string cleaning lambda
list_to_str = lambda lst: re.sub(r'[():\r\n]', '', ', '.join(lst))

# list-to-count cleaning lambda
list_to_count = lambda lst: len(lst)

# method for making flag vars from text string vars based on regex matching
# target_regex exmple: r'translat.*'
def flag_by_regex(target_regex, str_to_search_in):
    search_result = re.search(target_regex, str_to_search_in, re.IGNORECASE)
    if search_result:
        return 1
    else:
        return 0
    
    
# target_regex exmple: r'\d{4}'
def extract_by_regex(target_regex, str_to_search_in):
    search_result = re.search(target_regex, str_to_search_in, re.IGNORECASE)
    if search_result:
        return search_result.group(0)
    else:
        return ''

In [10]:
# encounter_id
# e.g.: [: 10056\r\n ]
df2['encounter_id'] = df2['encounter_id'].apply(list_to_str)

In [11]:
# individual_id
# e.g.: '460044\r\n'
df2['individual_id'] = df2['individual_id'].apply(lambda s: s.strip())

In [12]:
# occurrence_id -- fine as it is
df2['occurrence_id'][0]

'DFOC:Mon Aug 16 00:00:00 EDT 2004:N2:2'

In [13]:
# encounter_date
df2['encounter_date'] = pd.to_datetime(df2['encounter_date'].apply(lambda s: s.strip()))

In [14]:
## Location info
df2['location_name'] = df2['encounter_location_info'].apply(lambda a: a[0].strip())
df2['location_id'] = df2['encounter_location_info'].apply(lambda a: re.sub(r'[:\r\n]', '', a[2]).strip())
# Latitude 
df2['latitude'] = pd.to_numeric(df2['encounter_location_info'].apply(lambda a: a[5].strip()))
# Longitude  
df2['longitude'] = pd.to_numeric(df2['encounter_location_info'].apply(lambda a: a[8].strip()))

df2 = df2.drop(['encounter_location_info'], axis=1)

In [15]:
## More location fields
# locality - blank for over 2,000 obs, so not using it.
df2 = df2.drop(['locality'], axis=1)
# region_name
df2['region_name'] = df2['region_name'].apply(lambda a: a[1].strip())

In [16]:
## Surveyor details
# vessel
df2['vessel'] = df2['vessel'].apply(lambda a: a[1].strip())

In [17]:
# submitter
df2['submitter'] = df2['submitter'].apply(lambda a: a[1].strip())

In [18]:
# photographer
df2['photographer'] = df2['photographer'].apply(lambda a: a[1].strip())

In [19]:
## behavior
# behavior, beh_role
df2['behavior'] = df2['behavior'].apply(lambda a: a[1].strip())
df2['beh_role'] = df2['beh_role'].apply(lambda a: ''.join(a).strip())

In [20]:
## group behavior
# group_behavior, group_type
df2['group_behavior'] = df2['group_behavior'].apply(lambda a: ''.join(a).strip())
df2['group_type'] = df2['group_type'].apply(lambda a: ''.join(a).strip())

In [21]:
# best estimate of group size
df2['est_group_size'] = pd.to_numeric(df2['est_size_best'].apply(lambda a: ''.join(a).strip()))
df2 = df2.drop(['est_size_best'], axis=1)

In [22]:
## sighting -- need to clean these further
# sighting -- this field is not useful, looks like it was inconsistently used for different purposes by surveyors
# so I'm dropping it
df2 = df2.drop(['sighting'], axis=1)

#len(df2['sighting'].value_counts())

#df2['sighting'].value_counts()

In [23]:
df2['sighting_comments'] = df2['sighting_comments'].apply(lambda a: ''.join(a).strip())

In [24]:
len(df2['sighting_comments'].unique())

3416

In [25]:
df2['sighting_comments'].unique()

array(['TAIL LOBBING', '',
       'large feeding aggregation_  loose affiliations_  highly fluid groups',
       ...,
       "Both have white pecs_ Mn's would get 400 yds apart before coming back together_",
       'This humpback was mostly on the surface playing in a clump of kelp, poking his nose out and rolling, pec-slapping, etc.',
       'Whales too dispersed to effectively work from RHIB.'], dtype=object)

In [26]:
df2['sighting_comments'].value_counts()

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       11224
----                                                                                                                                                                                                                                                       

In [27]:
## comments
df2['additional_comments'] = df2['additional_comments'].apply(lambda a: ''.join(a).strip())

In [28]:
## whale description
df2['sex'] = df2['encounter_sex'].apply(lambda a: a.strip())
df2 = df2.drop(['encounter_sex'], axis=1)

In [29]:
# life_stage -- all empty
#df2['life_stage'].apply(lambda a: ''.join(a).strip()).value_counts()
df2 = df2.drop(['life_stage'], axis=1)

In [30]:
# scarring
def parse_scarring(s):
    s = s.strip()
    if s.lower() == 'none':
        n = 0
    elif s == '':
        n = np.nan
    else:
        n = 1
    return n

#df2['scarring'].apply(parse_scarring).value_counts(dropna=False)
df2['scarring_flag'] = df2['scarring'].apply(parse_scarring)
#df2[df2['scarring_flag'] == 1]
df2 = df2.drop(['scarring'], axis=1)

In [31]:
# status -- all whales in the set are marked as Alive in this field, so this field is not informative 
# and I'm dropping it

#df2['status'].apply(lambda a: a.strip()).value_counts(dropna=False)
df2 = df2.drop(['status'], axis=1)

In [32]:
# other

In [33]:
def parse_number_calves(a):
    s = ''.join(a).strip()
    s = s.strip('?')
    s = s.strip('+')
    if s.lower() == 'o':
        s = '0'
    if s.lower() == 'nd':
        s = ''
    return s

df2['number_calves'] = pd.to_numeric(df2['number_calves'].apply(parse_number_calves))
#pd.to_numeric(df2['number_calves'].apply(parse_number_calves)).value_counts(dropna=False)

In [34]:
# pos_type -- all empty
#df2['pos_type'].apply(lambda a: ''.join(a).strip()).value_counts()
df2 = df2.drop(['pos_type'], axis = 1)

In [35]:
df2.head(20)

,additional_comments,beh_role,behavior,encounter_date,encounter_id,group_behavior,group_type,individual_id,number_calves,occurrence_id,photographer,region_name,sighting_comments,submitter,url,vessel,location_name,location_id,latitude,longitude,est_group_size,sex,scarring_flag
0,,,Solo,2004-08-16 09:17:00,10059,Fluke Slap,Single,460044,0.0,DFOC:Mon Aug 16 00:00:00 EDT 2004:N2:2,John Calambokidis,Northern British Columbia,TAIL LOBBING,Department of Fisheries and Oceans Canada,http://www.splashcatalog.org:80/encounters/enc...,N2,British Columbia,NBC,52.542,-131.202,1.0,male,0.0
1,,,Unspecified competitive group member,2004-03-29 12:53:00,1006,Competitive,Competitive Group,420485,0.0,UABCS:Mon Mar 29 00:00:00 EST 2004:RA:6,,Mexico- Baja Calfiornia,,Universidad Autonoma de Baja California Sur,http://www.splashcatalog.org:80/encounters/enc...,RA,Baja California,MX-BC,23.388,-109.320,6.0,male,0.0
2,,AD,Unknown,2004-08-26 09:00:00,10056,Feeding,UNKN,472177,1.0,UAFK:Thu Aug 26 00:00:00 EDT 2004:DU:1,,Northern Gulf of Alaska,large feeding aggregation_ loose affiliations...,"University of Alaska, Fairbanks, Kodiak",http://www.splashcatalog.org:80/encounters/enc...,DU,Kodiak Island,NGOA,58.626,-152.303,40.0,male,0.0
3,,UN,Unknown,2004-08-26 09:00:00,10055,Feeding,UNKN,472380,1.0,UAFK:Thu Aug 26 00:00:00 EDT 2004:DU:1,,Northern Gulf of Alaska,large feeding aggregation_ loose affiliations...,"University of Alaska, Fairbanks, Kodiak",http://www.splashcatalog.org:80/encounters/enc...,DU,Kodiak Island,NGOA,58.626,-152.303,40.0,unknown,0.0
4,Mom?,G3,"Trio, non-competitive",2004-09-25 16:20:00,10044,,Mom calf Escort,450262,0.0,CRC:Sat Sep 25 00:00:00 EDT 2004:N2:6,John Calambokidis,California-Oregon,,Cascadia Research Collective,http://www.splashcatalog.org:80/encounters/enc...,N2,California,CA-OR,37.681,-122.863,3.0,unknown,0.0
5,,,Unknown,2004-10-20 13:30:00,10064,Feeding,,460031,0.0,DFOC:Wed Oct 20 00:00:00 EDT 2004:RR:2,Luciano Dalla Rosa,Northern British Columbia,one whale joined a single then they both appea...,Department of Fisheries and Oceans Canada,http://www.splashcatalog.org:80/encounters/enc...,RR,British Columbia,NBC,53.209,-129.052,2.0,unknown,0.0
6,,,Solo,2004-10-20 13:04:00,10063,Resting,Single,460313,0.0,DFOC:Wed Oct 20 00:00:00 EDT 2004:RR:1,Luciano Dalla Rosa,Northern British Columbia,,Department of Fisheries and Oceans Canada,http://www.splashcatalog.org:80/encounters/enc...,RR,British Columbia,NBC,53.209,-129.085,1.0,unknown,0.0
7,SPLASH ID updated February 2012 based on gentoype,UN,Unknown,2004-08-26 09:00:00,10054,Feeding,UNKN,700430,1.0,UAFK:Thu Aug 26 00:00:00 EDT 2004:DU:1,,Northern Gulf of Alaska,large feeding aggregation_ loose affiliations...,"University of Alaska, Fairbanks, Kodiak",http://www.splashcatalog.org:80/encounters/enc...,DU,Kodiak Island,NGOA,58.626,-152.303,40.0,female,0.0
8,,,Unknown,2004-10-18 09:12:00,10061,,,460179,0.0,DFOC:Mon Oct 18 00:00:00 EDT 2004:JPT:106,"John Ford, Mark Malleson",Northern British Columbia,,Department of Fisheries and Oceans Canada,http://www.splashcatalog.org:80/encounters/enc...,JPT,British Columbia,NBC,53.905,-130.833,NaN,unknown,0.0
9,,,Solo,2004-08-16 09:39:00,10060,Slow Travel,Single,460275,0.0,DFOC:Mon Aug 16 00:00:00 EDT 2004:N2:3-5,"John Calambokidis, Jessie Huggins",Northern British Columbia,,Department of Fisheries and Oceans Canada,http://www.splashcatalog.org:80/encounters/enc...,N2,British Columbia,NBC,52.542,-131.202,1.0,female,0.0


In [36]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19185 entries, 0 to 19184
Data columns (total 23 columns):
additional_comments    19185 non-null object
beh_role               19185 non-null object
behavior               19185 non-null object
encounter_date         19185 non-null datetime64[ns]
encounter_id           19185 non-null object
group_behavior         19185 non-null object
group_type             19185 non-null object
individual_id          19185 non-null object
number_calves          13670 non-null float64
occurrence_id          19185 non-null object
photographer           19185 non-null object
region_name            19185 non-null object
sighting_comments      19185 non-null object
submitter              19185 non-null object
url                    19185 non-null object
vessel                 19185 non-null object
location_name          19185 non-null object
location_id            19185 non-null object
latitude               19185 non-null float64
longitude              191

In [37]:
df2['individual_id'].value_counts()

420248    35
460293    27
420195    27
420183    25
460065    24
420270    24
420159    23
460336    22
420479    21
550159    21
460388    21
420560    20
420674    19
420412    19
420268    19
430520    19
420389    19
474377    18
540037    18
440144    18
420144    18
420464    18
420200    18
420656    18
550364    17
          ..
700542     1
520193     1
560261     1
520570     1
460013     1
700572     1
430135     1
472651     1
700557     1
472071     1
630551     1
640268     1
520326     1
620360     1
450208     1
430491     1
640296     1
420021     1
430435     1
470493     1
560179     1
630267     1
470916     1
474434     1
640065     1
Name: individual_id, Length: 8489, dtype: int64

In [38]:
df2[df2['individual_id'] == '420195']

,additional_comments,beh_role,behavior,encounter_date,encounter_id,group_behavior,group_type,individual_id,number_calves,occurrence_id,photographer,region_name,sighting_comments,submitter,url,vessel,location_name,location_id,latitude,longitude,est_group_size,sex,scarring_flag
596,dorsal matches to other sightings of T447 scra...,SE,CH/SE,2005-02-26 10:25:00,15438,,CG,420195,0.0,HSU:Sat Feb 26 00:00:00 EST 2005:PA:2,JJ,Mexico- Archipielago Revillagigedo,,Humboldt State University,http://www.splashcatalog.org:80/encounters/enc...,PA,Revillagigedo,MX-AR,18.729,-110.984,3.0,male,0.0
609,"no singers, no groups, windy, return to shore ...",MO,Mother with calf of year,2005-03-01 12:18:00,15443,,MCE,420195,1.0,HSU:Tue Mar 01 00:00:00 EST 2005:PA:4,JJ,Mexico- Archipielago Revillagigedo,"no singers, no groups, windy, return to shore ...",Humboldt State University,http://www.splashcatalog.org:80/encounters/enc...,PA,Revillagigedo,MX-AR,18.779,-111.037,3.0,male,0.0
891,,CH,CH/SE,2005-02-20 15:51:00,15975,,CG,420195,0.0,UNAM:Sun Feb 20 00:00:00 EST 2005:LO:2,SESA,Mexico- Archipielago Revillagigedo,,Universidad Nacional Autonoma de Mexico,http://www.splashcatalog.org:80/encounters/enc...,LO,Revillagigedo,MX-AR,18.861,-110.874,3.0,male,0.0
1291,,SE,CH/SE,2005-02-19 14:38:00,15306,,CG,420195,0.0,UNAM:Sat Feb 19 00:00:00 EST 2005:LO:7,HRN,Mexico- Archipielago Revillagigedo,,Universidad Nacional Autonoma de Mexico,http://www.splashcatalog.org:80/encounters/enc...,LO,Revillagigedo,MX-AR,18.848,-111.025,4.0,male,0.0
1916,"B1 of morning, called G7 in notes field confus...",S/J,Pair,2005-02-23 11:03:00,14166,,SJ,420195,0.0,HSU:Wed Feb 23 00:00:00 EST 2005:PA:1,JJ,Mexico- Archipielago Revillagigedo,,Humboldt State University,http://www.splashcatalog.org:80/encounters/enc...,PA,Revillagigedo,MX-AR,18.725,-110.974,2.0,male,0.0
1963,,CH,CH/SE,2005-02-13 12:00:00,14090,,CG,420195,0.0,UNAM:Sun Feb 13 00:00:00 EST 2005:LO:2,,Mexico- Archipielago Revillagigedo,,Universidad Nacional Autonoma de Mexico,http://www.splashcatalog.org:80/encounters/enc...,LO,Revillagigedo,MX-AR,18.712,-110.960,3.0,male,0.0
2494,,,Pair,2005-02-21 16:46:00,13326,,2,420195,0.0,UNAM:Mon Feb 21 00:00:00 EST 2005:LO:7,HRN,Mexico- Archipielago Revillagigedo,,Universidad Nacional Autonoma de Mexico,http://www.splashcatalog.org:80/encounters/enc...,LO,Revillagigedo,MX-AR,18.829,-111.018,2.0,male,0.0
2552,,SE,CH/SE,2005-03-03 10:18:00,13177,,CG,420195,0.0,UNAM:Thu Mar 03 00:00:00 EST 2005:LO:1,HRN,Mexico- Archipielago Revillagigedo,,Universidad Nacional Autonoma de Mexico,http://www.splashcatalog.org:80/encounters/enc...,LO,Revillagigedo,MX-AR,18.719,-110.920,3.0,male,0.0
2555,,SE,Unknown,2005-03-03 16:02:00,13179,,2,420195,0.0,UNAM:Thu Mar 03 00:00:00 EST 2005:LO:6,CALM,Mexico- Archipielago Revillagigedo,"MARIA, SOLO PIEL",Universidad Nacional Autonoma de Mexico,http://www.splashcatalog.org:80/encounters/enc...,LO,Revillagigedo,MX-AR,18.751,-111.001,2.0,male,0.0
3617,,NA,Nuclear Animal in a competitive group,2004-02-19 00:00:00,1084,,Competitive Group,420195,NaN,UNAM:Thu Feb 19 00:00:00 EST 2004:LO:7,ABV,Mexico- Archipielago Revillagigedo,Start Dec Lat/Long corrected based on GIS anal...,Universidad Nacional Autonoma de Mexico,http://www.splashcatalog.org:80/encounters/enc...,LO,Revillagigedo,MX-AR,18.715,-110.915,NaN,male,0.0


In [39]:
df2[df2['individual_id'] == '420248']['encounter_date'].dt.date.value_counts()

2005-03-29    3
2004-03-07    2
2004-02-25    2
2004-03-20    2
2004-03-23    2
2005-03-17    2
2006-03-23    2
2006-03-07    1
2004-03-11    1
2005-02-26    1
2005-03-31    1
2004-02-21    1
2006-03-15    1
2005-03-16    1
2005-03-12    1
2004-03-26    1
2005-04-06    1
2004-03-09    1
2005-02-23    1
2004-02-20    1
2004-03-22    1
2005-03-28    1
2006-04-02    1
2004-03-13    1
2006-02-24    1
2005-03-05    1
2004-03-12    1
Name: encounter_date, dtype: int64

In [41]:
ls ..

 Volume in drive C is Windows
 Volume Serial Number is 0AC5-9A85

 Directory of C:\Users\Arina\data_science\metis\project3

02/08/2018  02:58 PM    <DIR>          .
02/08/2018  02:58 PM    <DIR>          ..
02/08/2018  02:56 PM    <DIR>          .ipynb_checkpoints
02/08/2018  08:00 PM    <DIR>          data_local
02/10/2018  10:44 PM    <DIR>          project_mcnulty
               0 File(s)              0 bytes
               5 Dir(s)  586,979,639,296 bytes free


In [42]:
# Pickle the cleaned encounter-level dataset
df2.to_pickle('../data_local/whale_encounters_clean.pkl')